## LISA Demo

[LISA (Linux Integrated System Analysis)](https://lisa-linux-integrated-system-analysis.readthedocs.io/en/master/index.html) is a framework to record ftrace traces and analyze them as Panda dataframes.

This notebook shows a typical LISA-use case:

* Connects to the VM
* Collects a trace while executing a workload
* Displays the trace
* Analyses the trace

It can serve as a template for your own experiments.

## Target configuration

Connect to the VM started by VSCode using SSH at `localhost:5555` with a [Target](https://lisa-linux-integrated-system-analysis.readthedocs.io/en/master/target.html#lisa.target.Target) object.

In [ ]:
from lisa.target import Target
import os

target = Target(
    kind='linux',
    name='vscode-vm',
    host='localhost',
    port='5555',
    username='root',
    keyfile=os.path.expanduser('~/.ssh/vm-cli_rsa'),
    strict_host_check=False,
    kernel_src="../"
)

## Setting up a workload

Create a [RT App](https://lisa-linux-integrated-system-analysis.readthedocs.io/en/master/workloads.html#lisa.wlgen.rta.RTA), convenient for scheduler experiments.

In [ ]:
from lisa.wlgen.rta import RTA, RTAPhase, PeriodicWload

rtapp_profile = {
    f'tsk{cpu}_{i}': RTAPhase(
        prop_wload=PeriodicWload(
            duty_cycle_pct=20,
            period=16e-3,
            duration=1,
        )
    )
    for cpu in range(target.number_of_cpus)
    for i in range(3)
}

workload = RTA.from_profile(target, rtapp_profile, name='lisa_workload')

## Tracing the workload

We need to specify the ftrace events we want to record. You can find a list of ftrace events in `/sys/kernel/debug/tracing/available_events`.

In [ ]:
events = [
    'sched_switch',
    'sched_wakeup',
    'sched_wakeup_new',
    'task_rename',
]

Now let's record a trace while running the workload:

In [ ]:
from lisa.trace import FtraceCollector

trace_path = os.path.join(workload.res_dir, 'trace.dat')
ftrace_collector = FtraceCollector(target, events=events, buffer_size=10240, output_path=trace_path)

with workload, ftrace_collector:
    workload.run()

## Loading the trace

The [Trace](https://lisa-linux-integrated-system-analysis.readthedocs.io/en/master/trace_analysis.html#id1) class wraps a `trace.dat`.

In [ ]:
from lisa.trace import Trace

trace = Trace(trace_path, target.plat_info, events=events)

## Looking at the trace

If you have `kernelshark` installed, it can be opened with:

In [ ]:
# trace.show()

## Analysing the trace

### With data frames

In [ ]:
df = trace.df_event('sched_switch')
df

The [standard DataFrame operations](https://pandas.pydata.org/docs/user_guide/10min.html) are available, so you can filter/slice it however you wish:

In [ ]:
df[df.next_comm == 'systemd'].iloc[0:3]

#### With trace analysis functions

LISA ships [a number of trace analysis](https://lisa-linux-integrated-system-analysis.readthedocs.io/en/master/trace_analysis.html) methods under `trace.ana`.

In [ ]:
trace.ana.tasks.df_tasks_runtime()

In [ ]:
from lisa.trace import TaskID

df = trace.ana.tasks.df_task_states(TaskID(pid=0, comm='systemd'), stringify=True)
df

In [ ]:
from lisa.analysis.tasks import TaskState

df[df.curr_state == TaskState.TASK_ACTIVE]

### With plots

In [ ]:
import holoviews as hv

hv.extension('bokeh')

In [ ]:
trace.ana.tasks.plot_task_total_residency(TaskID(pid=0, comm="systemd"))

In [ ]:
trace.ana.tasks.plot_tasks_wakeups_heatmap(bins=200)

In [ ]:
trace.ana.tasks.plot_tasks_forks_heatmap(bins=200)